In [1]:
import sys
sys.path.append('../')
import nexullance.LP_optimal as LP_optimal
# import nexullance.LP_gurobi as LP_MP
import topologies.Slimfly as Slimfly
import globals as gl
import time
import tracemalloc
from statistics import mean
import networkx as nx
import pynauty as nauty

In [2]:
config = (128, 12)
topo_name='slimfly'
EPR=config[1]//2
_network = Slimfly.Slimflytopo(config[0], config[1])
V=config[0]
assert(V==_network.nx_graph.number_of_nodes())
network_edges=list(_network.nx_graph.edges())

In [3]:
p2p_TM=gl.generate_shift_traffic_pattern(config[0], EPR)
R2R_TM=gl.convert_p2p_traffic_matrix_to_R2R(p2p_TM, config[0], EPR)

In [4]:
#initialize adjacency dict to that of G_1
G_adjacency_dict={n: list(nbrdict.keys()) for n, nbrdict in _network.nx_graph.adjacency()}
vertex_weight={} #first a dictionary of vertex weights, later converted to a list of sets
G_2_edges=0
for i in range(V):
    for j in range(V):
        if R2R_TM[i][j] > 0:
            new_v_id=V+G_2_edges
            G_adjacency_dict[new_v_id]=[i]
            G_adjacency_dict[j].append(new_v_id)
            if R2R_TM[i][j] in vertex_weight.keys():
                assert(new_v_id not in vertex_weight[R2R_TM[i][j]])
                vertex_weight[R2R_TM[i][j]].append(new_v_id)
            else:
                vertex_weight[R2R_TM[i][j]]=[new_v_id]
            G_2_edges+=1

# convert vertex weight dict to a list of color sets
G_color_sets=[]
for color_set in vertex_weight.values():
    G_color_sets.append(set(color_set))

In [5]:
G=nauty.Graph(V+G_2_edges, directed=True, adjacency_dict=G_adjacency_dict, vertex_coloring=G_color_sets)

In [6]:
gen_set=gl.nauty_autgrp_verbose(G, _verbose=True)

Generators of the automorphism group:
[[1, 0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14, 17, 16, 19, 18, 21, 20, 23, 22, 25, 24, 27, 26, 29, 28, 31, 30, 33, 32, 35, 34, 37, 36, 39, 38, 41, 40, 43, 42, 45, 44, 47, 46, 49, 48, 51, 50, 53, 52, 55, 54, 57, 56, 59, 58, 61, 60, 63, 62, 65, 64, 67, 66, 69, 68, 71, 70, 73, 72, 75, 74, 77, 76, 79, 78, 81, 80, 83, 82, 85, 84, 87, 86, 89, 88, 91, 90, 93, 92, 95, 94, 97, 96, 99, 98, 101, 100, 103, 102, 105, 104, 107, 106, 109, 108, 111, 110, 113, 112, 115, 114, 117, 116, 119, 118, 121, 120, 123, 122, 125, 124, 127, 126, 129, 128, 131, 130, 133, 132, 135, 134, 137, 136, 139, 138, 141, 140, 143, 142, 145, 144, 147, 146, 149, 148, 151, 150, 153, 152, 155, 154, 157, 156, 159, 158, 161, 160, 163, 162, 165, 164, 167, 166, 169, 168, 171, 170, 173, 172, 175, 174, 177, 176, 179, 178, 181, 180, 183, 182, 185, 184, 187, 186, 189, 188, 191, 190, 193, 192, 195, 194, 197, 196, 199, 198, 201, 200, 203, 202, 205, 204, 207, 206, 209, 208, 211, 210, 213, 212, 

In [7]:
# apply automorphism on a vector
def apply_automorphism(vector, automorphism):
    new_vector=[]
    for v in vector:
        new_vector.append(automorphism[v])
    return new_vector
        

# find representative vector by the automorphism group
def representative_vector(vectors, generator_set):
    # inputs: 
    # 1. vectors: a list of vectors of the same size '_n', each vector is in the space of V^n, V is the number of vertices in G_1
    # 2. generator: generator set of the automorphism group
    num_vectors=len(vectors)
    _n=len(vectors[0])

    # outputs:
    # 1. representative vectors
    # 2. the exact automorphisms that maps all vectors to their representatives
    rep_vectors_to_all={} # keys: representative vectors, values: represented vectors

    # initialzing necessary data structures:
    orbit=set()
    checked_vectors=set()

    # start of the algorithm:
    for vector in vectors:
        if tuple(vector) in checked_vectors:
            continue
        representative=tuple(vector)
        rep_vectors_to_all[representative]=[]
        orbit.add(tuple(vector))
        while len(orbit)>0:
            _v=orbit.pop()
            checked_vectors.add(_v)
            for gen in generator_set:
                _w=apply_automorphism(_v, gen)
                if tuple(_w) not in checked_vectors:
                    orbit.add(tuple(_w))
                    rep_vectors_to_all[representative].append(_w)

    return rep_vectors_to_all



In [8]:
vectors=[]
for s in range(V):
    for d in range(V):
        for (i, j) in network_edges:
            vectors.append([s, d, i, j])
            vectors.append([s, d, j, i])

In [9]:
len(vectors)

25165824

In [10]:
representative_dict=representative_vector(vectors, gen_set)

In [11]:
len(representative_dict)

3145728

In [12]:
len(representative_dict)/len(vectors)

0.125